In [ ]:
def jaccard_precompute(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)]
    a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)]
    b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return jacc_index

In [ ]:
def create_MDS_perceptual_map(df, attrName, dissim):
    embedding = MDS(n_components=2, dissimilarity=dissim)
    mds_coord = embedding.fit_transform(df)
    fig, ax=plt.subplots(figsize=(10,5))
    ax.set_title(f'MDS Perceptual map: {attrName}', fontsize=20)
    plt.scatter(mds_coord[:,0],mds_coord[:,1])
    labels=df.index
    for label,x,y in zip(labels, mds_coord[:,0], mds_coord[:,1]):
        plt.annotate(label,(x,y), fontsize=13)
    plt.axhline(y=0, color='k', linestyle='-', linewidth=1)
    plt.axvline(x=0, color='k', linestyle='-', linewidth=1)
    plt.xlabel('PC1', fontsize=16)
    plt.ylabel('PC2', fontsize=16)
    plt.grid(True)
    plt.show()
    return mds_coord.shape, embedding

In [ ]:
# Co-occurrence dissimilarity (distance = 1-index) ใช้กับ MDS (อย่าใช้ similarity)
def gen_precomputed_matrix(df_b, df_attr):
    precomputed_matrix = pd.DataFrame(index=df_b.columns,columns=df_b.columns).fillna(0)
    
    jacc = pd.DataFrame(columns=df_attr.columns)
    jacc = jacc.append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)
    
    for i in range(0,len(precomputed_matrix.columns)):
        for j in range(0,len(precomputed_matrix.columns)):
            for k in range(0,len(df_attr.columns)): # 0 to 329
                try:
                    jacc.iloc[0,k] = 1-jaccard_precompute(df_b, df_attr,i, j, k)
                except Exception:
                    jacc.iloc[0,k] = None
                    continue
            precomputed_matrix.iloc[i,j] = jacc.mean(axis=1)[0]
            jacc = jacc.iloc[0:0].append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)
    return precomputed_matrix